# HSR Mujoco Example

HSR in Mujoco simulator.

### Dependencies:

1. [Giskard branch: mujoco_actions_devel](https://github.com/SemRoCo/giskardpy/tree/37b4048c3787c1f5f0370ca4e8ee10754eaa12c9)

1. [mujoco_sim](https://github.com/HoangGiang93/mujoco_sim)

1. [HSR description files](https://github.com/maltehue/mujoco_robots)

## 1. Launch Robot and envrionment

In [1]:
import os
import psutil
import subprocess
import rospy
from ipywidgets import Button, Layout, GridBox, VBox, FloatSlider, ToggleButtons, Box
from IPython.display import display, Markdown, IFrame
from sidecar import Sidecar

# Directory of the ROS launch files
LAUNCH_FILE_DIR = os.path.abspath(os.path.join(os.getcwd(), "../launch"))
# To display web UI on the left
SIDECAR = None
# To manage the roslaunch process in the background
LAUNCH_PROCESS = None

# UI widgets variables
UI_CONFIG = {
    'robots': [
        {
            'name': 'HSR',
            'value': '$(find hsr_mujoco)/model/hsrb4s.xml',
        },
        {
            'name': 'HSR with a cup',
            'value': '$(find hsr_mujoco)/model/hsrb4s_cup.xml',
        },
        {
            'name': 'HSR with a tray',
            'value': '$(find hsr_mujoco)/model/hsrb4s_tray.xml',
        }
    ],
    'worlds': [
        {
            'name': 'world_containers',
            'value': '$(find hsr_mujoco)/model/world_containers.xml',
        },
        {
            'name': 'world_only_containers',
            'value': '$(find hsr_mujoco)/model/world_only_containers.xml',
        },
        {
            'name': 'world_particle_container',
            'value': '$(find hsr_mujoco)/model/world_particle_container.xml',
        },
        {
            'name': 'iai_apartment',
            'value': '$(find mujoco_world)/model/iai_apartment/iai_apartment_with_window4.xml',
        },
        {
            'name': 'waterfront',
            'value': '$(find mujoco_world)/model/waterfront/world.xml',
        },
        {
            'name': 'iai_kitchen',
            'value': '$(find mujoco_world)/model/iai_kitchen/iai_kitchen_python.xml',
        },
    ]
}
SELECTED_ROBOT = UI_CONFIG['robots'][0]['value']
SELECTED_WORLD = UI_CONFIG['worlds'][0]['value']

try:
    DISPLAY_NUM = os.environ['DISPLAY']
except KeyError:
    DISPLAY_NUM = None

# Popup the xpra desktop on the left panel
def open_xpra():
    global SIDECAR
    if SIDECAR is not None:
        SIDECAR.close()
    SIDECAR = Sidecar(title='XRPA', anchor='right')
    try:
        JUPYTERHUB_USER = '/user/' + os.environ['JUPYTERHUB_USER']
    except KeyError:
        JUPYTERHUB_USER = ''
    xpra_url = f"{JUPYTERHUB_USER}/xprahtml5/index.html"
    print(xpra_url)
    with SIDECAR:
        display(IFrame(src=xpra_url, width='100%', height='100%'))

# Execute the roslaunch command
def launch_simulation():
    global LAUNCH_PROCESS
    if LAUNCH_PROCESS is not None:
        LAUNCH_PROCESS.terminate()
        LAUNCH_PROCESS.wait()
    if DISPLAY_NUM == ':100' or True:
        open_xpra()
    launchfile = os.path.join(LAUNCH_FILE_DIR, "hsr_mujoco.launch")
    command = [
        'roslaunch', 
        launchfile,
        'gui:=false',
        'web:=false',
        f"mujoco_robot:={SELECTED_ROBOT}",
        f"mujoco_world:={SELECTED_WORLD}"
    ]
    LAUNCH_PROCESS = psutil.Popen(command,
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL)
    print(f"""
        Executing command: 
        {' '.join(command)}
        in the background, the output will be hidden. 
        To check the output please execute it in a Terminial!
    """)

# Create UI widgets
def select_robot(index):
    global SELECTED_ROBOT
    SELECTED_ROBOT = UI_CONFIG['robots'][index]['value']

def select_world(index):
    global SELECTED_WORLD
    SELECTED_WORLD = UI_CONFIG['worlds'][index]['value']

def create_widgets(config):
    widgets = []
    robot_select = ToggleButtons(
        options=[i['name'] for i in config['robots']],
        description='robots',
        tooltips=[i['value'] for i in config['robots']],
    )
    robot_select.observe(lambda v: select_robot(v['new']), names="index")

    world_select = ToggleButtons(
        options=[i['name'] for i in config['worlds']],
        description='worlds',
        tooltips=[i['value'] for i in config['worlds']],
    )
    world_select.observe(lambda v: select_world(v['new']), names="index")

    Launch_btn = Button(
        description='Launch Simulation',
        layout=Layout(width='auto'),
        button_style='success'
    )
    Launch_btn.on_click(lambda b: launch_simulation())
    widgets.append(robot_select)
    widgets.append(world_select)
    widgets.append(Launch_btn)
    return VBox(widgets)

# Display widgets
display(create_widgets(UI_CONFIG))

## 2. Control the robot with the Giskard Python Interface

In [2]:
import rospy
import random
from giskardpy.python_interface.python_interface import GiskardWrapper
from giskardpy.goals.joint_goals import JointPositionList
from giskardpy.monitors.joint_monitors import JointGoalReached
from geometry_msgs.msg import PoseStamped, Point, Quaternion, Vector3Stamped, PointStamped, QuaternionStamped

rospy.init_node('notebook_playground')

def move_robot(gk_wrapper, pos, root_link='map', tip_link='base_link'):
    pos_stamp = PointStamped()
    pos_stamp.header.frame_id = root_link
    pos_stamp.point = pos
    gk_wrapper.add_default_end_motion_conditions()
    gk_wrapper.motion_goals.add_cartesian_position(
        root_link=root_link,
        tip_link=tip_link,
        goal_point=pos_stamp,
    )
    gk_wrapper.execute()

def rotate_robot(gk_wrapper, ori, root_link='map', tip_link='base_link'):
    ori_stamp = QuaternionStamped()
    ori_stamp.header.frame_id = root_link
    ori_stamp.quaternion = ori
    gk_wrapper.add_default_end_motion_conditions()
    gk_wrapper.motion_goals.add_cartesian_orientation(
        root_link=root_link,
        tip_link=tip_link,
        goal_orientation=ori_stamp,
    )
    gk_wrapper.execute()

def control_joint(gk_wrapper, joint_goal):
    gk_wrapper.motion_goals.add_joint_position(goal_state=joint_goal)
    gk_wrapper.add_default_end_motion_conditions()
    gk_wrapper.execute()

def list_links(gk_wrapper):
    return gk_wrapper.world.get_group_info(gk_wrapper.world.get_group_names()[0]).links

def list_joints(gk_wrapper):
    return gk_wrapper.world.get_controlled_joints(gk_wrapper.world.get_group_names()[0])


#### Init Giskard python API wrapper

In [ ]:
giskard_wrapper = GiskardWrapper()

#### Move robot around

In [ ]:
# List all the links
print(list_links(giskard_wrapper))

# Move the robot to a absolute position
move_robot(giskard_wrapper, Point(1.2, 0, 0))

# Rotate robot body (absolute Coordinate)
rotate_robot(giskard_wrapper, Quaternion(0, 0, -0.1, 1))

#### Control joints

In [ ]:
# Control joints
print(list_joints(giskard_wrapper))
control_joint(giskard_wrapper, {
    'wrist_roll_joint': -2,
    'torso_lift_joint': 0.2
})

In [3]:
from geometry_msgs.msg import Twist, Vector3
import rospy

rospy.init_node('notebook_playground')
cmd_vel_pub = rospy.Publisher('/hsrb4s/cmd_vel', Twist, queue_size=5)
cmd_vel_msg = Twist()

# slider for moving velocity
linear_x = FloatSlider(
    value=0,
    min=-1,
    max=1,
    step=0.1,
    description='Moving',
    orientation='vertical',
    readout=True,
    readout_format='.1f',
)

def on_linear_x_change(v):
    cmd_vel_msg.linear.x = v
    cmd_vel_pub.publish(cmd_vel_msg)

linear_x.observe(lambda v: on_linear_x_change(v['new']), names='value')

# slider for rotation velocity
angular_z = FloatSlider(
    value=0,
    min=-1,
    max=1,
    step=0.1,
    description='Rotation',
    readout=True,
    readout_format='.1f',
)

def on_angular_z_change(v):
    cmd_vel_msg.angular.z = -v
    cmd_vel_pub.publish(cmd_vel_msg)

angular_z.observe(lambda v: on_angular_z_change(v['new']), names='value')

display(Markdown('#### Robot Base Velocity Controller:'))
display(Box([linear_x, angular_z]))

#### Robot Base Velocity Controller:

Box(children=(FloatSlider(value=0.0, description='Moving', max=1.0, min=-1.0, orientation='vertical', readout_…